In [1]:
#imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm as tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [2]:
DATA_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split'

print(os.listdir(DATA_DIRECTORY))

['test', 'train', 'val']


In [3]:
TRAIN_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/train'
VAL_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/val'
TEST_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/test'

In [4]:
print(os.listdir(TRAIN_DIRECTORY))

['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']


In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Hyper-parameters 
num_epochs = 15
batch_size = 100
learning_rate = 0.01

In [7]:
train_data = ImageFolder(TRAIN_DIRECTORY, transform=ToTensor())
val_data = ImageFolder(VAL_DIRECTORY, transform=ToTensor())
test_data = ImageFolder(TEST_DIRECTORY, transform=ToTensor())

In [8]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

19075
2373
2416


In [9]:
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_dl = DataLoader(val_data, batch_size*2, num_workers=4, pin_memory=True)
val_dl = DataLoader(test_data, batch_size*2,shuffle=True, num_workers=4, pin_memory=True)

In [10]:
#load pretrained model
model = torchvision.models.vgg16_bn(pretrained = True)

In [11]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [12]:
for param in model.parameters():
    param.requires_grad = True

model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(in_features=512*3*3, out_features=256, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=256, out_features=128, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=128, out_features=35, bias=True))


model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [14]:
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/15], Step [100/191], Loss: 0.7474
Epoch [2/15], Step [100/191], Loss: 0.1579
Epoch [3/15], Step [100/191], Loss: 0.0666
Epoch [4/15], Step [100/191], Loss: 0.0544
Epoch [5/15], Step [100/191], Loss: 0.0386
Epoch [6/15], Step [100/191], Loss: 0.0294
Epoch [7/15], Step [100/191], Loss: 0.0262
Epoch [8/15], Step [100/191], Loss: 0.0249
Epoch [9/15], Step [100/191], Loss: 0.0158
Epoch [10/15], Step [100/191], Loss: 0.0138
Epoch [11/15], Step [100/191], Loss: 0.0137
Epoch [12/15], Step [100/191], Loss: 0.0119
Epoch [13/15], Step [100/191], Loss: 0.0097
Epoch [14/15], Step [100/191], Loss: 0.0114
Epoch [15/15], Step [100/191], Loss: 0.0097
Finished Training


In [15]:
def check_acc(dl):
    n_correct = 0
    n_samples = 0
    model.eval()

    with torch.no_grad():
        for images, labels in dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs, 1)

            n_correct += (predicted == labels).sum()
            n_samples += predicted.size(0)

        print(f"Val acc : Got {n_correct} / {n_samples} with accuracy {float(n_correct)/float(n_samples)*100:.2f}")

In [16]:
check_acc(train_dl)

Val acc : Got 19074 / 19075 with accuracy 99.99


In [17]:
check_acc(test_dl)

Val acc : Got 2365 / 2373 with accuracy 99.66


In [18]:
check_acc(val_dl)

Val acc : Got 2407 / 2416 with accuracy 99.63
